
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Deploying External Models using AI Gateway

In this demo, you will deploy an external model API using MLflow AI Gateway

**Learning Objectives**

*By the end of this demo, you will be able to:*

* Understand External Models API

* Benefits of centralized Management and Governance using Databricks Model Serving

## Setup Demo Environment

### Install dependencies and configure environment

In [0]:
!pip install mlflow[genai]>=2.9.0 databricks_genai_inference==0.2.3 openai==1.40.7
dbutils.library.restartPython()

Before starting the demo, run the provided classroom setup script.

In [0]:
%run ./Includes/Classroom-Setup

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

%md
### Pre-Requisite: Set up Secrets

To secure access to the serving endpoint, we need to set up a couple of secrets for the host (workspace URL) and a personal access token.

Secrets can be set up using the Databricks CLI with the following commands:

<br>

```
databricks secrets create-scope <scope-name>
databricks secrets put-secret --json '{
  "scope": "<scope-name>",
  "key": "<key-name>",    
  "string_value": "<value>"
}' 
```

Once this is set up, we will load the values into variables for the notebook using the Secrets utility in Databricks.


**Note🚨:** The **OPENAI API key** is already set up in the environment, so no additional configuration is required.

In [0]:
import os

# Set OpenAI API key
openai_api_key = dbutils.secrets.get("llm_scope", "openai_token")

### Set the Endpoint name

Set a unique endpoint name. Then, check if an endpoint with this name already exists. If it does, delete the existing endpoint to avoid conflicts. This process ensures that your deployment will not fail due to duplicate endpoint names.

In [0]:
import mlflow.deployments

# Generate the unique endpoint name
ext_ms_endpoint_name = f"{DA.unique_name('_')}_openai_endpoint"
client = mlflow.deployments.get_deploy_client("databricks")

try:
    # Attempt to get the endpoint to check if it exists
    ext_ms_endpoint = client.get_endpoint(ext_ms_endpoint_name)
    # If the endpoint exists, delete it
    client.delete_endpoint(ext_ms_endpoint_name)
    print(f"Endpoint {ext_ms_endpoint_name} deleted successfully.")
except Exception as e:
    # Handle the case where the endpoint does not exist or another error occurs
    print(f"Endpoint {ext_ms_endpoint_name} could not be found or another error occurred: {e}")

## Deploy OpenAI model

Below is code to deploy an external model using Databricks Model Serving.  The supported external models can be found in our [documentation](https://docs.databricks.com/en/generative-ai/external-models/index.html)

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")
client.create_endpoint(
    name=ext_ms_endpoint_name,
    config={
        "served_entities": [{
            "name": "openai-completions",
            "external_model": {
                "name": "gpt-3.5-turbo-instruct",
                "provider": "openai",
                "task": "llm/v1/completions",
                "openai_config": {
                    "openai_api_key_plaintext": openai_api_key
                }
            }
        }]
    }
)

## Query the Deployed Model via UI

More realistically, we can query the deployed model directly from our serving applications.

We can query the model directly in Databricks to confirm everything is working using the **Query endpoint** capability.


Follow below steps to query the endpoint.

* Navigate to the **Serving** section in the Databricks sidebar to access the Serving UI.
* Locate and select the model serving endpoint created above.
* On the endpoint's details page, click on **Query endpoint**.
* In the provided text box, enter the model input data in JSON format. If the model was logged with an input example, you can click Show Example to automatically load this input.
* Click **Send Request** to submit your query to the model.

Below are the screenshots for reference:

* **Query**

  ![query](files/images/get-started-generative-ai-engineering-1.0.1/query.png)


* **Query Endpoint**

  ![query_endpoint](files/images/get-started-generative-ai-engineering-1.0.1/query_endpoint.png)


This process will send your input data to the model and display the model's response in the UI.

Sample query:
`{"prompt": "What organization created you?"}`

## Conclusion
You learned how to deploy/integrate and externally hosted model in Databricks.  Enabling unified governance and management of models in your organization.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>